
# Sujet 2: Modélisation des interdépendances

**Quel est l’impact d’une baisse de la capacité de LNG sur le système en général et en particulier sur
le système électrique ?**

Pour aller plus loin : modéliser le stockage gaz

**Authors**: Lucas DEFRANCE, Quentin DEHAENE, Jules PAINDESSOUS--POETE

## Imports

In [84]:
# Necessary imports
using JuMP
using XLSX
using DataFrames
#use the solver you want
using HiGHS

### Extension of optimization workshop to whole year

In [85]:
Tmax = 8736; #optimization for 1 year
data_file_zootopia = "data/TP_empilement.xlsx";

#data for load and fatal generation
load = XLSX.readdata(data_file_zootopia, "TP", "C10:C8745");
wind = XLSX.readdata(data_file_zootopia, "TP", "D10:D8745");
solar = XLSX.readdata(data_file_zootopia, "TP", "E10:E8745");
hydro_fatal = XLSX.readdata(data_file_zootopia, "TP", "F10:F8745");
thermal_fatal = XLSX.readdata(data_file_zootopia, "TP", "G10:G8745");

#total of RES
Pres = wind + solar + hydro_fatal + thermal_fatal;

8736×1 Matrix{Int64}:
 2961
 2486
 2111
 1684
 1482
 1478
 1603
 1940
 2507
 3012
 3219
 3427
 3743
    ⋮
 4905
 4656
 4483
 4186
 3898
 3797
 3903
 3765
 3285
 3045
 3446
 3580

In [86]:
data_file = "data/Donnees_etude_de_cas_ETE305.xlsx";

#data for power plants
plant_name = vec(XLSX.readdata(data_file,"Parc électrique","A2:A24"));
nb_units = vec(XLSX.readdata(data_file,"Parc électrique","D2:D24"));
pmax_per_unit = vec(XLSX.readdata(data_file,"Parc électrique","E2:E24")); #MWh
pmin_per_unit = vec(XLSX.readdata(data_file,"Parc électrique","F2:F24")); #MWh
dmin = vec(XLSX.readdata(data_file, "Parc électrique", "G2:G24")); #hours
costs = vec(XLSX.readdata(data_file, "Parc électrique", "H2:H24")); #€/MWh

# Convert vectors to specified data types
plant_name = String.(plant_name)
nb_units = Int.(nb_units)
pmax_per_unit = Float64.(pmax_per_unit)
pmin_per_unit = Float64.(pmin_per_unit)
costs = Float64.(costs)

# Create DataFrame with specified data types for each column. This will be useful for creating the vectors
# later on as some plants have multiples units that all have to be modelled
df = DataFrame(
    plant_name = plant_name,
    nb_units = nb_units,
    pmax_per_unit = pmax_per_unit,
    pmin_per_unit = pmin_per_unit,
    dmin = dmin,
    costs = costs
);

# Generate the plant_type vector using array comprehension
plant_type = [i in [10,14,15] ? "thermal_fatal" :
              i == 18 ? "hydro_fatal" :
              i == 19 ? "hydro_lake" :
              i == 20 ? "hydro_step" :
              i in [21,22] ? "wind" :
              i == 23 ? "solar" :
              "thermal" for i in 1:n_row]
        
# Append the vector as a new column to the DataFrame
df[!, :plant_type] = plant_type

println(df)


23×7 DataFrame
 Row │ plant_name    nb_units  pmax_per_unit  pmin_per_unit  dmin  costs    plant_type    
     │ String        Int64     Float64        Float64        Any   Float64  String        
─────┼────────────────────────────────────────────────────────────────────────────────────
   1 │ Iconuc               2         900.0           300.0  24       12.0  thermal
   2 │ Tabarnuc             2         900.0           300.0  24       12.0  thermal
   3 │ NucPlusUltra         2        1300.0           330.0  24       12.0  thermal
   4 │  Gazby               1         390.0           135.0  4        40.0  thermal
   5 │  Pégaz               2         394.0           137.0  4        40.0  thermal
   6 │  Samagaz             1         430.0           151.0  4        40.0  thermal
   7 │  Omaïgaz             2         430.0           151.0  4        40.0  thermal
   8 │  Gastafiore          1         430.0           151.0  4        40.0  thermal
   9 │ Igaznodon            2          8

In [87]:
# Initialize empty arrays to store values for non-fatal thermal clusters
names_th = []
costs_th = []
Pmin_th = []
Pmax_th = []
dmin_th = []

# Filter the DataFrame based on the plant_type column
df_thermal = filter(row -> row.plant_type == "thermal", df)

# Loop through each row of the DataFrame
for (k, row) in enumerate(eachrow(df_thermal))
    
    # Iterate over each plant name and unit number in the row
    plant_name = row.plant_name
    for i in 1:row.nb_units
        # Construct the complete plant name with unit number and append it to names_th
        plant_name_with_unit = string(plant_name, "-", i)
        push!(names_th, plant_name_with_unit)
        
        # Append other values to respective arrays based on your requirements
        push!(costs_th, row.costs)
        push!(Pmin_th, row.pmin_per_unit)
        push!(Pmax_th, row.pmax_per_unit)
        push!(dmin_th, row.dmin)
    end
end

# Convert the vectors to 2D arrays
costs_th = hcat(costs_th)
Pmin_th = hcat(Pmin_th)
Pmax_th = hcat(Pmax_th)
dmin_th = hcat(dmin_th)

# Create a dictionnary to save non fatal thermal plants names. For e.g. if we want to known which plant is a 
# index n in costs_th, we just have to run ask for dict_th[n]
Nth = length(costs_th)
dict_th = Dict(i => names_th[i] for i in 1:Nth)

# This should read "Iconuc-1"
dict_th[1]


"Iconuc-1"

In [88]:
#data for hydro reservoir
Nhy = 1 #number of hydro generation units
Pmin_hy = zeros(Nhy);
Pmax_hy = df[19,:pmax_per_unit] *ones(Nhy); #MW
e_hy = 10^6*ones(Nhy) #MWh (We know we have 1TWh of cumulated capacity)
costs_hy = df[19,:costs]*ones(Nhy); #MWh


In [89]:
#costs
cth = repeat(costs_th', Tmax) #cost of thermal generation €/MWh
chy = repeat(costs_hy', Tmax) #cost of hydro generation €/MWh
cuns = 5000*ones(Tmax) #cost of unsupplied energy €/MWh
cexc = 0*ones(Tmax); #cost of in excess energy €/MWh

In [90]:
#data for STEP
Pmax_STEP = df[20,:pmax_per_unit] #MW
rSTEP = 0.75
max_stock_STEP = 201600; #MWh

In [ ]:
#############################
#create the optimization model
#############################
model = Model(HiGHS.Optimizer)

#############################
#define the variables
#############################
#thermal generation variables
@variable(model, Pth[1:Tmax,1:Nth] >= 0)
@variable(model, UCth[1:Tmax,1:Nth], Bin)
@variable(model, UPth[1:Tmax,1:Nth], Bin)
@variable(model, DOth[1:Tmax,1:Nth], Bin)
#hydro generation variables
@variable(model, Phy[1:Tmax,1:Nhy] >= 0)
#unsupplied energy variables
@variable(model, Puns[1:Tmax] >= 0)
#in excess energy variables
@variable(model, Pexc[1:Tmax] >= 0)
#weekly STEP variables
@variable(model, Pcharge_STEP[1:Tmax] >= 0)
@variable(model, Pdecharge_STEP[1:Tmax] >= 0)
@variable(model, stock_STEP[1:Tmax] >= 0)
#battery variables
#@variable(model, Pcharge_battery[1:Tmax] >= 0)
#@variable(model, Pdecharge_battery[1:Tmax] >= 0)
#@variable(model, stock_battery[1:Tmax] >= 0)
#
# #############################
#define the objective function
#############################
@objective(model, Min, sum(Pth.*cth)+sum(Phy.*chy)+Puns'cuns+Pexc'cexc)

#############################
#define the constraints
#############################
#balance constraint
@constraint(model, balance[t in 1:Tmax], sum(Pth[t,g] for g in 1:Nth) + sum(Phy[t,h] for h in 1:Nhy) + Pres[t] + Puns[t] - load[t] - Pexc[t] - Pcharge_STEP[t] + Pdecharge_STEP[t]== 0)
#thermal unit Pmax constraints
@constraint(model, max_th[t in 1:Tmax, g in 1:Nth], Pth[t,g] <= Pmax_th[g]*UCth[t,g])
#thermal unit Pmin constraints
@constraint(model, min_th[t in 1:Tmax, g in 1:Nth], Pmin_th[g]*UCth[t,g] <= Pth[t,g])
#thermal unit Dmin constraints
for g in 1:Nth
        if (dmin[g] > 1)
            @constraint(model, [t in 2:Tmax], UCth[t,g]-UCth[t-1,g]==UPth[t,g]-DOth[t,g],  base_name = "fct_th_$g")
            @constraint(model, [t in 1:Tmax], UPth[t]+DOth[t]<=1,  base_name = "UPDOth_$g")
            @constraint(model, UPth[1,g]==0,  base_name = "iniUPth_$g")
            @constraint(model, DOth[1,g]==0,  base_name = "iniDOth_$g")
            @constraint(model, [t in dmin[g]:Tmax], UCth[t,g] >= sum(UPth[i,g] for i in (t-dmin[g]+1):t),  base_name = "dminUPth_$g")
            @constraint(model, [t in dmin[g]:Tmax], UCth[t,g] <= 1 - sum(DOth[i,g] for i in (t-dmin[g]+1):t),  base_name = "dminDOth_$g")
            @constraint(model, [t in 1:dmin[g]-1], UCth[t,g] >= sum(UPth[i,g] for i in 1:t), base_name = "dminUPth_$(g)_init")
            @constraint(model, [t in 1:dmin[g]-1], UCth[t,g] <= 1-sum(DOth[i,g] for i in 1:t), base_name = "dminDOth_$(g)_init")
    end
end

#hydro unit constraints
@constraint(model, unit_bounds_hy[t in 1:Tmax, h in 1:Nhy], Pmin_hy[h] <= Phy[t,h] <= Pmax_hy[h])
#hydro stock constraint
@constraint(model, stock_bounds_hy[h in 1:Nhy], sum(Phy[t,h] for t in 1:Tmax) <= e_hy[h])

#weekly STEP
@constraint(model, max_Pcharge_STEP[t in 1:Tmax],Pcharge_STEP[t] <= Pmax_STEP)
@constraint(model, max_Pdecharge_STEP[t in 1:Tmax],Pdecharge_STEP[t] <= Pmax_STEP)

@constraint(model, stock_STEP_max[t in 1:Tmax], stock_STEP[t] <= max_stock_STEP)
@constraint(model, stock_STEP_evo[t in 2:Tmax],stock_STEP[t] == stock_STEP[t-1] + Pcharge_STEP[t]*rSTEP - Pdecharge_STEP[t])
@constraint(model, stock_STEP_init[t in 1:Tmax],stock_STEP[1] == 0.5*max_stock_STEP)

#battery
#@constraint(model, max_Pcharge_battery[t in 1:Tmax], Pcharge_battery[t] <= Pmax_battery)
#@constraint(model, max_Pdecharge_battery[t in 1:Tmax], Pdecharge_battery[t] <= Pmax_battery)

#@constraint(model, stock_battery_max[t in 1:Tmax], stock_battery[t] <= Pmax_battery*d_battery)
#@constraint(model, stock_battery_evo[t in 2:Tmax],stock_battery[t] == stock_battery[t-1] + Pcharge_battery[t]*rbattery - Pdecharge_battery[t]/rbattery)
#@constraint(model, stock_battery_init[t in 1:Tmax],stock_battery[1] == 0)



#no need to print the model when it is too big
#solve the model
optimize!(model)
#------------------------------
#Results
@show termination_status(model)
@show objective_value(model)
#@show value.(stock_battery[t] for t in 1:Tmax)

#exports results as csv file
th_gen = value.(Pth)
hy_gen = value.(Phy)
STEP_charge = value.(Pcharge_STEP)
STEP_decharge = value.(Pdecharge_STEP)
#battery_charge = value.(Pcharge_battery)
#battery_decharge = value.(Pdecharge_battery)


# new file created
touch("results.csv")

# file handling in write mode
f = open("results.csv", "w")

for name in names
    write(f, "$name ;")
end
write(f, "Hydro ; STEP pompage ; STEP turbinage ; Batterie injection ; Batterie soutirage ; RES ; load ; Net load \n")

for t in 1:Tmax
    for g in 1:Nth
        write(f, "$(th_gen[t,g]) ; ")
    end
    for h in 1:Nhy
        write(f, "$(hy_gen[t,h]) ;")
    end
    write(f, "$(STEP_charge[t]) ; $(STEP_decharge[t]) ;")
    write(f, "$(battery_charge[t]) ; $(battery_decharge[t]) ;")
    write(f, "$(Pres[t]) ;  $(load[t]) ; $(load[t]-Pres[t]) \n")

end

close(f)